In [102]:
import pandas as pd

In [103]:
import numpy as np

In [104]:
from glob import glob

In [105]:
import itertools

In [106]:
import pygsheets as pyg

In [107]:
import time

#### some pandas opts

In [108]:
pd.options.display.max_columns = 100

In [109]:
pd.options.display.max_rows = 500

#### authorize pyg and set retry limit

In [110]:
gc = pyg.authorize(outh_file="./client_secret_986841527205-ndf6fbsfmrhjinnofm47j6olvmm11smn.apps.googleusercontent.com.json",
                   outh_nonlocal=True,
                   retries=5)

#### title for the google sheet and xls

In [111]:
sheet_name = "BY-nup133-day70"

#### xls output directory

In [112]:
xls_out_dir = "/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/BY-nup133-day70/"

#### where are the files?

In [113]:
%ls /home/dizak/Pulpit/BIONAS/G148/SNPs_calling/

16 documents from consensus.geneious*
16 documents from WT-day70 to WT-zero point.geneious*
BY-day0/
BY-nup133-day70/
BY-WT-day70/
C2_merged_dfs.csv*
consensus/
mapped_contigs/
mapped_reads/
nup mapped to wt cons.geneious*
W303-cog7-day42/
W303-nup133-day42/
W303-WT-day42/
WT-day70/
WT-day70.xlsx*


In [114]:
sampling_levels = ["N1", "N2", "N3"]

In [115]:
strain_gene_day_N1_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/BY-nup133-day70/N1/*csv")

In [116]:
strain_gene_day_N2_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/BY-nup133-day70/N2/*csv")

In [117]:
strain_gene_day_N3_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/BY-nup133-day70/N3/*csv")

#### get non-redundant list of genes in the inputfiles

In [118]:
def find_flat_value(inputfiles_list,
                    col_name = "CDS"):
    """
    Get flat list of desired values from list of CSV files.
    
    Parameters
    -------
    inputfiles_list: list of str
        List of input CSV files.
    col_name: str
        Desired column name in the input CSV file.
    
    Returns
    -------
    list of desired values.
    """
    values_list = []
    for i in inputfiles_list:
        df = pd.read_csv(i)
        if len(df) == 0:
            pass
        elif col_name not in df.columns:
            pass
        else:
            values_list.append(df[col_name].dropna().drop_duplicates().tolist())
    return list(itertools.chain.from_iterable(values_list))

#### get values from input files by key

In [119]:
def find_by_key(inputfiles_list,
                key):
    """
    Get pandas.DataFrame selected by a given key from list of CSV files.
    
    Parameters
    -------
    inputfiles_list: list of str
        List of input CSV files.
    key: str, int, float, bool
        Key used as query against rows in the CSV files.
    value_col: str
        Column name which holds values to be returned.
    
    Returns
    -------
    dict of lists of desired values if pandas.Dataframe not empty
    None if pandas.DataFrame empty
    """
    values_list = []
    for i in inputfiles_list:
        filename = "".join(i.split("/")[-1].split(".")[:-1])
        df = pd.read_csv(i)
        if len(df) == 0:
            pass
        else:
            if isinstance(key, str) == True:
                df_dtype_sel = df.select_dtypes(include=["object"])
            elif isinstance(key, int) == True:
                df_dtype_sel = df.select_dtypes(include=["int"])
            elif isinstance(key, float) == True:
                df_dtype_sel = df.select_dtypes(include=["float"])
            elif isinstance(key, bool) == True:
                df_dtype_sel = df.select_dtypes(include=["bool"])
            else:
                raise ValueError("key must str, int, float or bool dtype")
            for col in df_dtype_sel.columns:
                df_sel = df[df_dtype_sel[col] == key]
                if len(df_sel) > 0:
                    return {"dataframe": df_sel,
                            "filename": filename}

#### get whole set of pandas.DataFrames selections in one dict

In [120]:
def get_dfs_set(key_list,
                files_list,
                vals=["Minimum",
                      "Maximum",
                      "Change"],
                df_key="dataframe",
                key_index_name="Gene",
                smpl_index_name="Sample",
                row_index_name="Number",
                index_by_key=True,
                smpl_index_val=None,):
    """
    Get desired values in pandas.Dataframe gathered in dict by
    the list of keys.
    
    Parameters
    -------
    key_list: list, tuple
        List of keys by which pandas.Dataframes are
        initially selected.
    files_list: list, tuple
        List of input CSV files.
    vals: list of str, default: ["Minimum", 
                                 "Maximum",
                                 "Change"]
        Columns names holding data in pandas.DataFrames
        selected.
    df_key: str, default: <"dataframe">
        Key for generic pandas.DataFrame selection for
        SNPs-sheets_merge.find_by_key function.
    key_index_name: str, default: <"Gene">
        Name for index of selection key.
    smpl_index_name: str, default: <"Sample">
        Name for index of sample.
    row_index_name: str, default: <"Number">
        Name for numeric row index.
    index_by_key: bool, default: True
        Enables multiindexing.
    smpl_index_val: str, default: <None>
        Adds sample name to multiindexing if not <None>
    """
    out_dict = {}
    for i in key_list:
        key_vals = find_by_key(files_list,
                               key=i)[df_key]
        out_dict[i] = key_vals[vals]
    if index_by_key is True:
        for i in out_dict:
            key_index = [i] * len(out_dict[i])
            if smpl_index_val is not None:
                smpl_index = [smpl_index_val] * len(out_dict[i])
                tpls = list(zip(*[key_index,
                                  smpl_index,
                                  out_dict[i].index]))
                mindex = pd.MultiIndex.from_tuples(tpls,
                                                   names=[key_index_name,
                                                          smpl_index_name,
                                                          row_index_name])
            else:
                tpls = list(zip(*[key_index,
                                  out_dict[i].index]))
                mindex = pd.MultiIndex.from_tuples(tpls,
                                                   names=[key_index_name,
                                                          row_index_name])
            out_dict[i].index = mindex
    return out_dict

#### merge any given number of dfs

In [121]:
def merge_dfs(dfs,
              sort_cols=["Minimum",
                         "Maximum"],
              reconstr_index=True):
    """
    Merge any number of pandas.DataFrame into one.
    Indexes must be identical in all the pandas.DataFrames.
    
    Parameters
    -------
    dfs: list
        list of pandas.DataFrames to merge.
    sort_cols: list, None
        list of col names to sort the final
        pandas.DataFrame by. No sorting if None.
    """
    for x in [set(i.index) for i in dfs]:
        assert len(x) == 1, "Indices are not homogenic."
    new_index = list(x)
    df = reduce(lambda df1, df2: pd.merge(left=df1,
                                          right=df2,
                                          how="outer"),
                dfs)
    if sort_cols is not None:
        df.sort(columns=sort_cols)
    if reconstr_index is True:
        df.index = len(df) * new_index
    return df

#### let's find out which CDS are present in all the files one by one

In [122]:
strain_gene_day_N1_CDSs = find_flat_value(strain_gene_day_N1_files)
strain_gene_day_N2_CDSs = find_flat_value(strain_gene_day_N2_files)
strain_gene_day_N3_CDSs = find_flat_value(strain_gene_day_N3_files)

#### let's gather some info about each of the CDS from each sample

In [123]:
des_cols = ["Change",
            "Protein Effect",
            "Polymorphism Type",
            "Variant Frequency",
            "Average Quality",
            "Minimum",
            "Maximum"]

In [124]:
strain_gene_day_N1_dfs = get_dfs_set(strain_gene_day_N1_CDSs,
                                     strain_gene_day_N1_files,
                                     vals=des_cols,
                                     smpl_index_val=sampling_levels[0])
strain_gene_day_N2_dfs = get_dfs_set(strain_gene_day_N2_CDSs,
                                     strain_gene_day_N2_files,
                                     vals=des_cols,
                                     smpl_index_val=sampling_levels[1])
strain_gene_day_N3_dfs = get_dfs_set(strain_gene_day_N3_CDSs,
                                     strain_gene_day_N3_files,
                                     vals=des_cols,
                                     smpl_index_val=sampling_levels[2])

#### let's unwind them all from this dict into lists

In [125]:
N1_dfs = [strain_gene_day_N1_dfs[i] for i in strain_gene_day_N1_dfs.keys()]
N2_dfs = [strain_gene_day_N2_dfs[i] for i in strain_gene_day_N2_dfs.keys()]
N3_dfs = [strain_gene_day_N3_dfs[i] for i in strain_gene_day_N3_dfs.keys()]

#### let's merge them to see changes between samples.
#### rememeber now cannot use the merge_dfs function since multindex is NOT homogenic

In [126]:
N1_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
                N1_dfs)

In [127]:
N2_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
               N2_dfs)

In [128]:
N3_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
               N3_dfs)

In [129]:
N1_N2_N3_df = reduce(lambda df1, df2: pd.concat([df1, df2]),
                     [N1_df, N2_df, N3_df])

In [130]:
N1_N2_N3_df

Change Protein Effect  \
Gene                     Sample Number                                
MTL1 CDS                 N1     0             M -> C           None   
DAN4 CDS                 N1     4             T -> G   Substitution   
                                5             T -> G   Substitution   
                                6             C -> G   Substitution   
SWI1 CDS                 N1     8             W -> A     Truncation   
                                9             M -> A     Truncation   
                                10            M -> A     Truncation   
                                11            W -> A     Truncation   
                                12            W -> A     Truncation   
                                13            W -> A     Truncation   
                                14            W -> A   Substitution   
                                15          MK -> AT   Substitution   
                                16          WW -> TT   Substitution   
                                17        SAW -> GAT   Substitution   
FLO11 CDS                N1     0             C -> G   Substitution   
                                1             C -> G   Substitution   
                                2             G -> T           None   
                                3             G -> A   Substitution   
hypothetical protein CDS N1     6             T -> G           None   
                                7           CA -> TG   Substitution   
                                8       AACA -> TCAT   Substitution   
                                9           GT -> CA   Substitution   
                                10      CTAA -> ATTT   Substitution   
FLO9 CDS                 N1     2             G -> A           None   
                                3             C -> T   Substitution   
                                4             T -> A           None   
                                5             G -> A           None   
                                6           AG -> GC   Substitution   
                                7         GCA -> ATG   Substitution   
                                8             C -> T   Substitution   
                                9           TG -> CA   Substitution   
                                10            C -> T   Substitution   
                                11            T -> A           None   
                                12            G -> A           None   
                                13            S -> C   Substitution   
                                14            Y -> T   Substitution   
IRA2 CDS                 N1     0       (G)3 -> (G)4    Frame Shift   
MDM1 CDS                 N1     3             A -> T   Substitution   
SAN1 CDS                 N1     0             C -> T   Substitution   
NUP133 CDS               N1     0             G -> T           None   
                                1           GC -> AT   Substitution   
                                2         GTT -> TTA   Substitution   
                                3         AAA -> CTT   Substitution   
                                4         TAA -> ACT      Extension   
                                5             T -> A   Substitution   
                                6           CT -> TC   Substitution   
                                7             G -> C   Substitution   
                                8           AG -> TC   Substitution   
                                9         ATT -> GAG   Substitution   
                                10          TA -> AC   Substitution   
                                11          TT -> GC   Substitution   
                                12        TGG -> GGA   Substitution   
                                13            G -> T   Substitution   
                                14            T -> A   Substitution   
                                15            A -> G   Substitution   

#### let's make xls

In [29]:
writer = pd.ExcelWriter("{}/{}.xls".format(xls_out_dir, sheet_name))

In [30]:
for i in set(strain_gene_day_N1_CDSs +
             strain_gene_day_N2_CDSs +
             strain_gene_day_N3_CDSs):
    N1_N2_N3_df.xs(i).to_excel(excel_writer=writer,
                               sheet_name=i)

In [31]:
writer.save()

#### let's make sheet on gdrive

In [32]:
sh = gc.create(sheet_name)

In [33]:
for i in set(strain_gene_day_N1_CDSs +
             strain_gene_day_N2_CDSs +
             strain_gene_day_N3_CDSs):
    time.sleep(3)
    wks = sh.add_worksheet(i)
    wks.set_dataframe(N1_N2_N3_df.xs(i),
                      (1, 1),
                      copy_index=True,
                      fit=True)